A continuación te presento **la Clase 4 completa** **diseñada como un cuaderno Jupyter (`.ipynb`)**, **lista para copiar, pegar, ejecutar y publicar en GitHub**, coherente con el enfoque del curso y **articulada directamente con Stata** (conceptualmente) pero **implementada en Python**.



He estructurado el cuaderno **tal como debe verse pedagógicamente**:
👉 *celdas Markdown + celdas de código + actividades + proyectos*.

---



# Clase 4 – 6 de enero de 2026

## **Construcción de variables clave para el estudio de la desigualdad social**

### *Clases sociales, género, origen étnico y cohortes de edad*

---



# Objetivo de la clase



Que el estudiante sea capaz de:

* Construir **variables independientes centrales** para el estudio de la desigualdad social
* Comprender que dichas variables **no “vienen dadas”**, sino que son **construcciones teóricas**
* Implementar estas construcciones usando **Python como alternativa abierta a Stata**
* Preparar los datos para análisis bivariado y modelos estadísticos posteriores

---



# 1. Preparación del entorno de trabajo


In [ ]:


import pandas as pd
import numpy as np



# Crear dataset sintético para esta clase  



A continuación te dejo **el script de Python completo, corregido y robusto** para **crear el dataset sintético `encuesta_clase3.dta`**, exactamente **alineado con la Clase 3 y que sirve directamente como insumo de la Clase 4**.

Este script:

* Genera una **encuesta social sintética realista**
* Incorpora **desigualdades estructurales explícitas**
* Produce un archivo **compatible 100 % con Stata**
* Evita problemas de versiones recientes de `pandas`

---



# Script de Python

## Generación del dataset `encuesta_clase3.dta`

- **Ejecutar en Jupyter o Google Colab**


In [1]:
import pandas as pd
import numpy as np

# ==================================================
# 1. Reproducibilidad
# ==================================================
np.random.seed(2026)

# ==================================================
# 2. Tamaño de la muestra
# ==================================================
n = 300

# ==================================================
# 3. Variables sociodemográficas básicas
# ==================================================

df = pd.DataFrame({
    "id": range(1, n + 1),

    # Edad
    "edad": np.random.randint(18, 65, size=n),

    # Género (codificado)
    # 1 = Mujer, 2 = Hombre
    "genero": np.random.choice([1, 2], size=n, p=[0.52, 0.48]),

    # Nivel educativo
    # 1 = Primaria, 2 = Secundaria, 3 = Técnica, 4 = Universitaria
    "educacion": np.random.choice(
        [1, 2, 3, 4],
        size=n,
        p=[0.30, 0.34, 0.18, 0.18]
    ),

    # Clase social
    # 1 = Baja, 2 = Media, 3 = Alta
    "clase_social": np.random.choice(
        [1, 2, 3],
        size=n,
        p=[0.45, 0.40, 0.15]
    ),

    # Origen étnico
    # 1 = No étnico, 2 = Indígena, 3 = Afrodescendiente
    "origen_etnico": np.random.choice(
        [1, 2, 3],
        size=n,
        p=[0.68, 0.12, 0.20]
    ),

    # Condición de ocupación
    # 1 = Ocupado, 0 = No ocupado
    "ocupado": np.random.choice([0, 1], size=n, p=[0.30, 0.70])
})

# ==================================================
# 4. Ingreso: variable dependiente central
#    (con desigualdad estructural incorporada)
# ==================================================

ingreso_base = 700_000

efecto_educacion = df["educacion"] * 550_000
efecto_clase = df["clase_social"] * 650_000
penalizacion_genero = np.where(df["genero"] == 1, -280_000, 0)
penalizacion_etnica = np.where(df["origen_etnico"] != 1, -220_000, 0)
bono_ocupado = np.where(df["ocupado"] == 1, 300_000, 0)

ruido = np.random.normal(0, 350_000, size=n)

df["ingreso"] = (
    ingreso_base
    + efecto_educacion
    + efecto_clase
    + penalizacion_genero
    + penalizacion_etnica
    + bono_ocupado
    + ruido
).astype(int)

# Asegurar ingresos positivos
df["ingreso"] = df["ingreso"].clip(lower=600_000)

# ==================================================
# 5. Conversión a variables categóricas con etiquetas
#    (compatible con pandas modernos)
# ==================================================

df["genero"] = (
    df["genero"]
    .astype("category")
    .cat.rename_categories(["Mujer", "Hombre"])
)

df["educacion"] = (
    df["educacion"]
    .astype("category")
    .cat.rename_categories(["Primaria", "Secundaria", "Técnica", "Universitaria"])
)

df["clase_social"] = (
    df["clase_social"]
    .astype("category")
    .cat.rename_categories(["Baja", "Media", "Alta"])
)

df["origen_etnico"] = (
    df["origen_etnico"]
    .astype("category")
    .cat.rename_categories(["No étnico", "Indígena", "Afrodescendiente"])
)

df["ocupado"] = (
    df["ocupado"]
    .astype("category")
    .cat.rename_categories(["No ocupado", "Ocupado"])
)

# ==================================================
# 6. Guardar dataset en formato Stata
# ==================================================

df.to_stata("encuesta_clase3.dta", write_index=False)

# ==================================================
# 7. Verificación rápida
# ==================================================
df.head()


,id,edad,genero,educacion,clase_social,origen_etnico,ocupado,ingreso
0,1,19,Mujer,Técnica,Media,Afrodescendiente,Ocupado,3646875
1,2,24,Mujer,Universitaria,Media,No étnico,No ocupado,4371353
2,3,44,Hombre,Primaria,Media,No étnico,No ocupado,2203061
3,4,31,Hombre,Secundaria,Media,No étnico,No ocupado,3783735
4,5,31,Mujer,Primaria,Alta,Indígena,Ocupado,2888008



# ✅ ¿Qué garantiza este dataset?

✔️ Compatible con **Clase 3** (análisis descriptivo)
✔️ Insumo directo de la **Clase 4** (construcción de variables)
✔️ Realista para **regresión lineal y logística**
✔️ Desigualdades **no triviales**, visibles desde descriptivos
✔️ Totalmente reproducible

---



# Uso inmediato



### En Python


In [2]:
df = pd.read_stata("encuesta_clase3.dta")
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   id             300 non-null    int32   
 1   edad           300 non-null    int32   
 2   genero         300 non-null    category
 3   educacion      300 non-null    category
 4   clase_social   300 non-null    category
 5   origen_etnico  300 non-null    category
 6   ocupado        300 non-null    category
 7   ingreso        300 non-null    int32   
dtypes: category(5), int32(3)
memory usage: 5.8 KB



### En Stata

```stata
use encuesta_clase3.dta, clear
describe
tab clase_social genero
summarize ingreso
```

---



# Nota pedagógica para el curso

Este dataset **no es “neutro”**:
las desigualdades están **programadas explícitamente**, lo que permite discutir:

> “¿Qué parte del resultado es estructura y qué parte es azar?”




# 2. Carga del dataset de la encuesta


In [3]:


df = pd.read_stata("encuesta_clase3.dta")
df.head()



,id,edad,genero,educacion,clase_social,origen_etnico,ocupado,ingreso
0,1,19,Mujer,Técnica,Media,Afrodescendiente,Ocupado,3646875
1,2,24,Mujer,Universitaria,Media,No étnico,No ocupado,4371353
2,3,44,Hombre,Primaria,Media,No étnico,No ocupado,2203061
3,4,31,Hombre,Secundaria,Media,No étnico,No ocupado,3783735
4,5,31,Mujer,Primaria,Alta,Indígena,Ocupado,2888008



*Nota metodológica:*
Este dataset es una **simulación de una encuesta social**, pero reproduce estructuras reales de datos muestrales.

---



# 3. Marco conceptual: ¿qué es una variable de estratificación?

> Las variables de estratificación social no son simples atributos individuales.
> Son **expresiones empíricas de relaciones sociales estructurales**.

En este curso trabajaremos con:

* Clase social
* Estatus ocupacional
* Género
* Origen étnico
* Cohortes de edad

---



# 4. Variable género


In [4]:

df["genero"].value_counts(normalize=True) * 100



genero
Mujer     50.0
Hombre    50.0
Name: proportion, dtype: float64


## Discusión guiada

* ¿El género es una variable individual o estructural?
* ¿Qué desigualdades puede capturar en modelos posteriores?

---



# Variable binaria para modelos


In [ ]:


df["mujer"] = np.where(df["genero"] == "Mujer", 1, 0)
df[["genero", "mujer"]].head()




# *Equivalente Stata:*

```stata
gen mujer = genero == "Mujer"
```





# 5. Construcción de clase social (variable central)



## Clase social original


In [6]:


df["clase_social"].value_counts()



clase_social
Media    131
Baja     120
Alta      49
Name: count, dtype: int64


## Recodificación teórica (clase trabajadora vs no trabajadora)


In [7]:


df["clase_binaria"] = df["clase_social"].replace({
    "Baja": "Trabajadora",
    "Media": "No trabajadora",
    "Alta": "No trabajadora"
})

df["clase_binaria"].value_counts()



C:\Users\marco\AppData\Local\Temp\ipykernel_20460\4123663130.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df["clase_binaria"] = df["clase_social"].replace({


clase_binaria
No trabajadora    180
Trabajadora       120
Name: count, dtype: int64


# **Discusión clave**

> Esta recodificación es una **decisión teórica**, no técnica.




# 6. Estatus ocupacional


In [8]:


df["ocupado"].value_counts(normalize=True) * 100



ocupado
Ocupado       68.666667
No ocupado    31.333333
Name: proportion, dtype: float64


## Variable dummy


In [9]:


df["ocupado_bin"] = np.where(df["ocupado"] == "Ocupado", 1, 0)




# 7. Origen étnico como eje de desigualdad


In [10]:


pd.crosstab(df["origen_etnico"], df["clase_social"], normalize="index") * 100



clase_social,Baja,Media,Alta
origen_etnico,,,
No étnico,36.231884,46.859903,16.908213
Indígena,58.333333,27.777778,13.888889
Afrodescendiente,42.105263,42.105263,15.789474



📌 **Lectura sociológica**

* ¿Existen patrones de desigualdad étnico-clasistas?
* ¿Cómo se podrían modelar estas diferencias?

---



# 8. Cohortes de edad

## Construcción de cohortes


In [14]:
df["cohorte_edad"] = pd.cut(
    df["edad"],
    bins=[18, 29, 44, 64],
    labels=["18-29", "30-44", "45-64"]
)

df["cohorte_edad"].value_counts()



cohorte_edad
45-64    128
30-44    100
18-29     64
Name: count, dtype: int64


📌 *Equivalente Stata:*

```stata
egen cohorte = cut(edad), at(18,30,45,65)
```

---



# 9. Análisis univariado de variables construidas


In [ ]:


df[["ingreso"]].describe()



In [11]:


df.groupby("clase_binaria")["ingreso"].mean()



C:\Users\marco\AppData\Local\Temp\ipykernel_20460\287146467.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("clase_binaria")["ingreso"].mean()


clase_binaria
Trabajadora       2.558058e+06
No trabajadora    3.467531e+06
Name: ingreso, dtype: float64


📌 *Pregunta clave:*

> ¿Qué desigualdad aparece ya en el análisis descriptivo?

---



# 10. Análisis bivariado inicial



## Clase social y género


In [12]:
pd.crosstab(df["clase_binaria"], df["genero"], normalize="index") * 100


genero,Mujer,Hombre
clase_binaria,,
Trabajadora,45.833333,54.166667
No trabajadora,52.777778,47.222222



## Cohorte de edad e ingreso


In [15]:


df.groupby("cohorte_edad")["ingreso"].mean()



C:\Users\marco\AppData\Local\Temp\ipykernel_20460\2012139193.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("cohorte_edad")["ingreso"].mean()


cohorte_edad
18-29    3.046067e+06
30-44    3.099865e+06
45-64    3.127428e+06
Name: ingreso, dtype: float64


# 11. Actividad práctica individual (en clase)

## Tarea 1

1. Construir una nueva variable de estratificación
2. Justificar teóricamente la decisión
3. Mostrar su distribución

  - Entrega: Notebook con comentarios

---



# 12. Proyecto corto grupal (para la semana)

## Proyecto: *“Desigualdad social en datos de encuesta”*



Cada grupo debe:

1. Seleccionar **dos ejes de desigualdad**
2. Construir variables derivadas
3. Analizar desigualdad descriptiva
4. Proponer una hipótesis explicativa



# Resultados esperados:

* Tablas
* Medias comparadas
* Interpretación sociológica

---



##  13. Cierre conceptual de la clase

> Las variables no existen antes del análisis.
> **Se construyen desde la teoría y se validan con datos.**

Esta clase prepara directamente:

* Regresión lineal (Clases 5–6)
* Regresión logística (Clases 7–8)

---



# Preparación para la Clase 5

* Pensar:

  > ¿Cómo explicar el ingreso a partir de una sola variable?
* Lectura sugerida:

  * Introducción a la regresión lineal simple

---



# Recomendación final para GitHub

Guarda este cuaderno como:

```text
clase_4_construccion_variables_estratificacion.ipynb
```

---
